# Clean up existing features and engineer new ones

In [1]:
import dateutil
import datetime

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = 200

In [2]:
train = pd.read_csv('data/train.csv')
test  = pd.read_csv('data/test.csv')

## Exploring the data

In [3]:
train_nulls = train.isnull().sum()
test_nulls = test.isnull().sum()

nulls = pd.concat([train_nulls[train_nulls != 0], test_nulls[test_nulls != 0]], axis=1)
nulls.columns = ['train', 'test']

display(nulls)

,train,test
funder,3635,869
installer,3655,877
subvillage,371,99
public_meeting,3334,821
scheme_management,3877,969
scheme_name,28166,7092
permit,3056,737


In [4]:
df = pd.concat([train, test])

In [5]:
number_of_train_rows = train.shape[0]

In [6]:
display(
    train.shape,
    test.shape,
    df.shape,
    df.columns.tolist(),
)

(59400, 40)

(14850, 40)

(74250, 40)

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [7]:
suspect_features = ['wpt_name', 'subvillage', 'lga', 'ward', 'district_code',
                    'recorded_by', 'scheme_name', 'funder', 'installer'
                   ]

print('Number of factor levels\n')
for f in suspect_features:
    print('{:5d} - {}'.format(df[f].value_counts().size, f))

Number of factor levels

45684 - wpt_name
21425 - subvillage
  125 - lga
 2098 - ward
   20 - district_code
    1 - recorded_by
 2868 - scheme_name
 2140 - funder
 2410 - installer


In [8]:
features_to_drop = set([
    'wpt_name',    # too many factor levels
    'subvillage',  # too many factor levels and we have lat/long for the location
    'ward',        # too many factor levels and we have lat/long for the location
    'recorded_by', # constant
    'scheme_name', # too many factor levels and NaNs
    
    'num_private', # irrelevant info
    'region_code', # duplicate info (region)
])

## Reduce factor levels

In [9]:
def reduce_factor_levels(df, column_name, min_count=700, other_name='Other'):
    counts = df[column_name].value_counts()
    top = counts[counts >= min_count].index
    df.loc[~df[column_name].isin(top), column_name] = other_name

In [10]:
reduce_factor_levels(df, 'funder')
reduce_factor_levels(df, 'installer')

## Extract new features from dates

In [11]:
features_to_drop.add('date_recorded')

In [12]:
years       = df['date_recorded'].apply(lambda x: dateutil.parser.parse(x).year)
df['month'] = df['date_recorded'].apply(lambda x: dateutil.parser.parse(x).month)

In [13]:
def month_to_rain_season(x):
    if x <= 2:
        return 'dry short'
    elif x <= 5:
        return 'wet long'
    elif x <= 9:
        return 'dry long'
    return 'wet short'

In [14]:
df['rain_season'] = df['month'].apply(month_to_rain_season)

In [15]:
is_zero_year = df['construction_year'] == 0
known_years = df[~is_zero_year]['construction_year']
med_year = known_years.median()
min_year = known_years.min()
med_year, min_year

(2000.0, 1960)

In [16]:
df.loc[is_zero_year, 'construction_year'] = med_year
df['construction_year'] = (df['construction_year'] - min_year).astype(int)

In [17]:
df['estimated_age'] = years - df['construction_year']
df.loc[df.estimated_age < 0, 'estimated_age'] = 0

In [18]:
the_beginning = dateutil.parser.parse('1960-01-01')
df['days_since_the_beginning'] =\
    df['date_recorded'].apply(lambda x: (dateutil.parser.parse(x) - the_beginning).days)

In [19]:
df['construction_period'] = (df['construction_year'] / 5).astype(int) - 196*2

## Clean up categorical features

In [20]:
def clean_values(df, col, values_dict):
    for k, v in values_dict.items():
        df.loc[df[col] == k, col] = v

### Extraction type

In [21]:
df.groupby(['extraction_type_class','extraction_type_group'])['extraction_type'].value_counts()

extraction_type_class  extraction_type_group  extraction_type          
gravity                gravity                gravity                      33263
handpump               afridev                afridev                       2208
                       india mark ii          india mark ii                 3029
                       india mark iii         india mark iii                 135
                       nira/tanira            nira/tanira                  10205
                       other handpump         other - swn 81                 284
                                              other - play pump              101
                                              walimi                          60
                                              other - mkulima/shinyanga        2
                       swn 80                 swn 80                        4588
motorpump              mono                   mono                          3628
                       other motorpum

In [22]:
clean_values(df, 'extraction_type',
    {
        'india mark ii'             : 'india',
        'india mark iii'            : 'india',
        'other - swn 81'            : 'swn',
        'swn 80'                    : 'swn',
        'other - play pump'         : 'other handpump',
        'walimi'                    : 'other handpump',
        'other - mkulima/shinyanga' : 'other handpump',
        'cemo'                      : 'other motorpump',
        'climax'                    : 'other motorpump',
    }
)

In [23]:
features_to_drop.add('extraction_type_group')

In [24]:
df.groupby(['extraction_type_class'])['extraction_type'].value_counts()

extraction_type_class  extraction_type  
gravity                gravity              33263
handpump               nira/tanira          10205
                       swn                   4872
                       india                 3164
                       afridev               2208
                       other handpump         163
motorpump              mono                  3628
                       other motorpump        149
other                  other                 8102
rope pump              other - rope pump      572
submersible            submersible           5982
                       ksb                   1790
wind-powered           windmill               152
Name: extraction_type, dtype: int64

### Management

In [25]:
df.groupby(['management_group'])['management'].value_counts()

management_group  management      
commercial        private operator     2504
                  water authority      1123
                  company               859
                  trust                 105
other             other                1083
                  other - school        126
parastatal        parastatal           2229
unknown           unknown               683
user-group        vwc                 50624
                  wug                  8108
                  water board          3688
                  wua                  3118
Name: management, dtype: int64

In [26]:
df['scheme_management'].fillna('Unknown', inplace=True)

In [27]:
df['scheme_management'].value_counts()

VWC                 45917
WUG                  6496
Unknown              4846
Water authority      3975
WUA                  3551
Water Board          3462
Parastatal           2124
Company              1341
Private operator     1326
Other                 996
SWC                   123
Trust                  92
None                    1
Name: scheme_management, dtype: int64

In [28]:
df.loc[df['scheme_management'] == 'None', 'scheme_management'] = 'Unknown'

### Quality

In [29]:
df.groupby(['quality_group'])['water_quality'].value_counts()

quality_group  water_quality     
colored        coloured                623
fluoride       fluoride                244
               fluoride abandoned       23
good           soft                  63505
milky          milky                  1005
salty          salty                  6082
               salty abandoned         423
unknown        unknown                2345
Name: water_quality, dtype: int64

In [30]:
features_to_drop.add('quality_group')

### Source

In [31]:
df.groupby(['source_class', 'source_type'])['source'].value_counts()

source_class  source_type           source              
groundwater   borehole              machine dbh             13822
                                    hand dtw                 1108
              shallow well          shallow well            21140
              spring                spring                  21216
surface       dam                   dam                       840
              rainwater harvesting  rainwater harvesting     2863
              river/lake            river                   11964
                                    lake                      950
unknown       other                 other                     261
                                    unknown                    86
Name: source, dtype: int64

In [32]:
df.loc[df['source'] == 'other', 'source'] = 'unknown'

In [33]:
features_to_drop.add('source_type')

### Waterpoint

In [34]:
df.groupby(['waterpoint_type_group'])['waterpoint_type'].value_counts()

waterpoint_type_group  waterpoint_type            
cattle trough          cattle trough                    150
communal standpipe     communal standpipe             35628
                       communal standpipe multiple     7611
dam                    dam                                8
hand pump              hand pump                      21884
improved spring        improved spring                  959
other                  other                           8010
Name: waterpoint_type, dtype: int64

In [35]:
features_to_drop.add('waterpoint_type_group')

### Payment

In [36]:
df.groupby(['payment_type'])['payment'].value_counts()

payment_type  payment              
annually      pay annually              4570
monthly       pay monthly              10397
never pay     never pay                31712
on failure    pay when scheme fails     4842
other         other                     1314
per bucket    pay per bucket           11266
unknown       unknown                  10149
Name: payment, dtype: int64

In [37]:
features_to_drop.add('payment')

### Quantity

In [38]:
df.groupby(['quantity_group'])['quantity'].value_counts()

quantity_group  quantity    
dry             dry              7782
enough          enough          41522
insufficient    insufficient    18896
seasonal        seasonal         5075
unknown         unknown           975
Name: quantity, dtype: int64

In [39]:
features_to_drop.add('quantity_group')

In [40]:
df.groupby(['quality_group'])['water_quality'].value_counts()

quality_group  water_quality     
colored        coloured                623
fluoride       fluoride                244
               fluoride abandoned       23
good           soft                  63505
milky          milky                  1005
salty          salty                  6082
               salty abandoned         423
unknown        unknown                2345
Name: water_quality, dtype: int64

In [41]:
features_to_drop.add('quality_group')

### Location info

In [42]:
df.groupby(['region'])['region_code'].value_counts()

region         region_code
Arusha         2              3709
               24              402
Dar es Salaam  7              1020
Dodoma         1              2779
Iringa         11             6599
Kagera         18             4174
Kigoma         16             3533
Kilimanjaro    3              5494
Lindi          80             1536
               8               375
               18                9
Manyara        21             1972
Mara           20             2451
Mbeya          12             5758
Morogoro       5              5038
Mtwara         90             1133
               99              512
               9               499
Mwanza         19             3824
               17               72
               12                1
Pwani          6              2032
               60             1298
               40                1
Rukwa          15             2242
Ruvuma         10             3306
Shinyanga      17             6262
               14           

In [43]:
enc = LabelEncoder()
df['region_code'] = enc.fit_transform(df['region']) + 1

In [44]:
for i in range(1, 22):
    enc = LabelEncoder()
    ith_reg = df['region_code'] == i
    df.loc[ith_reg, 'district_code'] = enc.fit_transform(df[ith_reg]['district_code']) + 1

In [45]:
df['district_code'] = df['region_code'] * 100 + df['district_code']
df['district_code'] = df['district_code'].astype(str)

In [46]:
def impute_mean_geo(df, col):
    has_value = df[col] != 0
    
    mean_dis = df[has_value].groupby(['district_code'])[col].mean()
    reg14_long = df[(df['region_code'] == 14) & (df['longitude'] != 0)]['longitude'].mean()
    
    df.loc[~has_value, col] = df[~has_value]['district_code']\
        .apply(lambda x: mean_dis[x] if x in mean_dis.index else reg14_long)

In [47]:
impute_mean_geo(df, 'latitude')
impute_mean_geo(df, 'longitude')

In [48]:
def extract_lga(x):
    if 'rural' in x:
        return 'rural'
    elif 'urban' in x:
        return 'urban'
    return 'other'

In [49]:
df['lga'] = df['lga'].apply(extract_lga)

## Remaining features

In [50]:
features_to_drop.add('amount_tsh')

In [51]:
df.fillna('unknown', inplace=True)
objects = df.select_dtypes(include=['O']).columns.values
df[objects] = df[objects].apply(lambda x: x.astype(str).str.lower())

## Drop useless

In [52]:
df.drop(features_to_drop, axis=1, inplace=True)

## Write to csv

In [53]:
df[0:number_of_train_rows].to_csv('data/cleaned_train.csv')
df[number_of_train_rows:].to_csv('data/cleaned_test.csv')

In [54]:
df.columns.tolist()

['id',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'basin',
 'region',
 'district_code',
 'lga',
 'population',
 'public_meeting',
 'scheme_management',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_class',
 'management',
 'management_group',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'source_class',
 'waterpoint_type',
 'month',
 'rain_season',
 'estimated_age',
 'days_since_the_beginning',
 'construction_period']